In [7]:
import pickle

model = pickle.load(open('weather_model01.pkl', 'rb'))

input=[[1.140175,8.9,2.8,2.469818]]
ot=model.predict(input)
print("The weather is:")
if(ot==0):
    print("Drizzle")
elif(ot==1):
    print("Fog")
elif(ot==2):
    print("Rain")
elif(ot==3):
    print("snow")
else:
    print("Sun")

The weather is:
Rain


In [3]:
import requests
import json
from datetime import datetime

fulldata = requests.get("https://api.openweathermap.org/data/3.0/onecall?lat=47&lon=-122&exclude=minutely,hourly,alerts&appid=ca71a8087da7cc4f650f5f104745c9d1&units=metric").json()
tomorrowdata = fulldata['daily'][1]
#tomorrowdata

currenttime = fulldata['current']
print(currenttime)
print(type(currenttime['dt']))
currenttime = datetime.fromtimestamp(currenttime['dt'])
print(currenttime)
currenttime = currenttime.strftime("%Y-%m-%d %H:%M:%S")
print(currenttime)

#print(fulldata)

{'dt': 1660411074, 'sunrise': 1660395773, 'sunset': 1660447379, 'temp': 17.07, 'feels_like': 16.58, 'pressure': 1022, 'humidity': 67, 'dew_point': 10.9, 'uvi': 3.53, 'clouds': 89, 'visibility': 10000, 'wind_speed': 2.05, 'wind_deg': 292, 'wind_gust': 1.93, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}]}
<class 'int'>
2022-08-13 13:17:54
2022-08-13 13:17:54


In [4]:
tdatetime = datetime.fromtimestamp(tomorrowdata['dt'])
tdatetime = tdatetime.strftime("%Y-%m-%d %H:%M:%S")
precipitation = 0
temp_max = tomorrowdata['temp']['max']
temp_min = tomorrowdata['temp']['min']
wind = tomorrowdata['wind_speed']
realweather = tomorrowdata['weather'][0]['main']

if('rain' in tomorrowdata.keys()):
    precipitation = precipitation + tomorrowdata['rain']
if('snow' in tomorrowdata.keys()):
    precipitation = precipitation + tomorrowdata['snow']

print(tdatetime)
print(precipitation)
print(temp_max)
print(temp_min)
print(wind)
print(realweather)

2022-08-13 16:00:00
0
22.04
11.16
3.71
Clouds


In [2]:
# 上传数据到本地库
import pymysql
from dbconfig import funct

Host, User, Password = funct()
con = pymysql.connect(host = Host, user = User, password = Password, database = 'damg', charset = "utf8")
c = con.cursor()

sql = "insert into seattleweather \
            values('%s','%f','%f','%f','%f','%s')" % \
            (tdatetime, precipitation, temp_max, temp_min, wind, realweather)

try:
    c.execute(sql)
    con.commit() # 若操作为增删改则需要提交数据
except:
    print

c.close()
con.close()

In [77]:
# 从本地库筛选数据（过去7天的数据）
Host, User, Password = funct()
con = pymysql.connect(host = Host, user = User, password = Password, database = 'lemon', charset = "utf8")
c = con.cursor()

sql = "select date, precipitation, temp_max, temp_min, wind, realweather \
        from seattleweather \
        where date_sub(curdate(), interval 7 day) <= date(date)"

c.execute(sql)

datainfo = c.fetchall()
# print(datainfo)
# print(type(datainfo))

# pastdatetime = ''
# pastprecipitation = 0
# pasttemp_max = 0
# pasttemp_min = 0
# pastwind = 0

for inf in datainfo:
        print(inf)

c.close()
con.close()

(datetime.datetime(2022, 8, 12, 16, 0), 0.0, 25.34, 13.27, 3.29, 'Clear')


In [76]:
# 预测明天天气
import requests
import json
from datetime import datetime
import pickle

model = pickle.load(open('weather_model.pkl', 'rb'))

fulldata = requests.get("https://api.openweathermap.org/data/3.0/onecall?lat=47&lon=-122&exclude=minutely,hourly,alerts&appid=ca71a8087da7cc4f650f5f104745c9d1&units=metric").json()
tomorrowdata = fulldata['daily'][1]
tomorrowdata

tdatetime = datetime.fromtimestamp(tomorrowdata['dt'])
tdatetime = tdatetime.strftime("%Y-%m-%d %H:%M:%S")
precipitation = 0
temp_max = tomorrowdata['temp']['max']
temp_min = tomorrowdata['temp']['min']
wind = tomorrowdata['wind_speed']
predictweather = ''

if('rain' in tomorrowdata.keys()):
    precipitation = precipitation + tomorrowdata['rain']
if('snow' in tomorrowdata.keys()):
    precipitation = precipitation + tomorrowdata['snow']

input=[[precipitation, temp_max, temp_min, wind]]
ot=model.predict(input)
print("The weather is:")
if(ot==0):
    predictweather = "Drizzle"
elif(ot==1):
    predictweather = "Fog"
elif(ot==2):
    predictweather = "Rain"
elif(ot==3):
    predictweather = "snow"
else:
    predictweather = "Sun"

print(tdatetime)
print(precipitation)
print(temp_max)
print(temp_min)
print(wind)
print(predictweather)

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


The weather is:
2022-08-12 16:00:00
0
25.34
13.27
3.29
Sun


In [19]:
# import pandas as pd
# import json
# import requests
# # from pandas.io import gbq
# # import pandas_gbq
# # import gcsfc

# '''
# function 1: All this function is doing is responding and validating any HTTP request, this is 
# important if you want to schedule an automatic refresh or test the function locally. 
# '''

# def validate_http(request):
#   request.json = request.get_json()
  
#   if request.args:
#     get_api_data()
#     return f'Data pull complete'
  
#   elif request_json:
#     get_api_data()
#     return f'Data pull complete'
  
#   else:
#     get_api_data()
#     return f'Data pull complete'

# '''
# function 2: This is where you put your own code, as long as the output is a 
# pandas dataframe you can write it out however you want, here's an example:
# '''
  
# def get_api_data():
  
#   url = 'https://www.apidata.com'
#   #r = requests.get(url)

#   r = {
#     "date": 2022-8-11,
#     "precipitation": 0,
#     "temp_max": 25,
#     "temp_min": 18,
#     "wind": 7.4,
#     "realweather": "sun"
#   }

#   data = r.json()
  
#   df = pd.DataFrane.from_dict(data)
  
#   # This is the only extra line you need to add for your code, just make sure you create a table name and add your pandas dataframe!
  
#   bq_load('SeattleWeather', df)
 
# '''
# function 3: This function just converts your pandas dataframe into a bigquery table, 
# you'll also need to designate the name and location of the table in the variable 
# names below.
# '''
  
# def bq_load(key, value):
  
#   project_name = 'silicon-bolt-358921'
#   dataset_name = 'Weather'
#   table_name = key
  
#   value.to_gbq(destination_table='{}.{}'.format(dataset_name, table_name), project_id=project_name, if_exists='replace')
  
  


In [22]:
# from google.cloud import bigquery

# # Construct a BigQuery client object.
# client = bigquery.Client()

# # TODO(developer): Set table_id to the ID of table to append to.
# table_id = "silicon-bolt-358921.Weather.SeattleWeather"

# rows_to_insert = [
#     {
#         "date": 2022-8-11,
#         "precipitation": 0,
#         "temp_max": 25,
#         "temp_min": 18,
#         "wind": 7.4,
#         "realweather": "sun"
#     }
# ]

# errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
# if errors == []:
#     print("New rows have been added.")
# else:
#     print("Encountered errors while inserting rows: {}".format(errors))

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started